# 1. Install prerequisites

In [ ]:
from importlib.metadata import version

In [1]:
version('torch')

'2.4.0'

In [5]:
version('triton')

'3.0.0'

In [ ]:
pip install --upgrade transformers

In [2]:
version('transformers')

'4.44.2'

## vLLM

https://docs.vllm.ai/en/latest/

In [ ]:
pip install --upgrade vllm

In [3]:
version('vllm')

'0.6.1.post2'

## SGLang

https://sglang.readthedocs.io/en/latest/

In [ ]:
pip install --upgrade "sglang[all]"

In [4]:
version('sglang')

'0.3.1.post2'

In [ ]:
pip install flashinfer -i https://flashinfer.ai/whl/cu124/torch2.4/

In [6]:
version('flashinfer')

'0.1.6+cu124torch2.4'

## ollama

https://github.com/ollama/ollama/blob/main/docs/linux.md#manual-install

In [11]:
!mkdir ollama && curl -L https://ollama.com/download/ollama-linux-amd64.tgz -o ollama-linux-amd64.tgz && tar -C ./ollama -xzf ollama-linux-amd64.tgz && rm ollama-linux-amd64.tgz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   117  100   117    0     0    566      0 --:--:-- --:--:-- --:--:--   567
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1583M  100 1583M    0     0  17.1M      0  0:01:32  0:01:32 --:--:-- 16.9M


```bash
./ollama/bin/ollama serve &
```

https://github.com/ollama/ollama-python

https://github.com/ollama/ollama/tree/main/docs

In [ ]:
pip install --upgrade ollama

In [9]:
version('ollama')

'0.3.3'

# 2. Performance testing functions

## Test models

https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct

https://huggingface.co/collections/neuralmagic/llama-31-quantization-66a3f907f48d07feabb8f300

In [1]:
test_models = {                                                                    # OpenLLM leaderboard score
    "llama-3.1" : "meta-llama/Meta-Llama-3.1-8B-Instruct",                         # 100.0 %
    "llama-3.1:w8a16" : "neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a16",  # 99.8 %    
    "llama-3.1:fp8" : "neuralmagic/Meta-Llama-3.1-8B-Instruct-FP8",                # 99.5 % - warning the "FP8-dynamic" version is MUCH slower on RTX 4090 !
    "llama-3.1:w8a8" : "neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a8",    # 99.4 %
    "llama-3.1:w4a16" : "neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w4a16",  # 97.1 %
    "qwen-2.5" : "Qwen/Qwen2.5-7B-Instruct",
    "qwen-2.5:w8a16" : "Qwen/Qwen2.5-7B-Instruct-GPTQ-Int8",
    "qwen-2.5:w4a16" : "Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4",
    "qwen-2.5-14b:w8a16" : "Qwen/Qwen2.5-14B-Instruct-GPTQ-Int8",
    "qwen-2.5-14b:w4a16" : "Qwen/Qwen2.5-14B-Instruct-GPTQ-Int4",
    "qwen-2.5-32b:w4a16" : "Qwen/Qwen2.5-32B-Instruct-GPTQ-Int4",    
    "qwen-2.5-coder" : "Qwen/Qwen2.5-Coder-7B-Instruct",
    "qwen-2.5-coder:fp8" : "Qwen2.5-Coder-7B-Instruct-FP8-Dynamic"
}

Note: to create custom quantized model versions -> https://docs.vllm.ai/en/latest/quantization/fp8.html#quantization-process

https://ollama.com/library/llama3.1

```bash
./ollama/bin/ollama pull llama3.1:8b-instruct-fp16
./ollama/bin/ollama pull llama3.1:8b-instruct-q8_0
./ollama/bin/ollama pull llama3.1:8b-instruct-q4_0

./ollama/bin/ollama pull qwen2.5:7b-instruct-fp16
./ollama/bin/ollama pull qwen2.5:7b-instruct-q8_0
./ollama/bin/ollama pull qwen2.5:7b-instruct-q4_K_M
./ollama/bin/ollama pull qwen2.5:14b-instruct-q4_K_M
./ollama/bin/ollama pull qwen2.5:32b-instruct-q4_K_M

./ollama/bin/ollama pull qwen2.5-coder:7b-instruct-q8_0
./ollama/bin/ollama pull qwen2.5-coder:7b-instruct-q4_K_M
```

In [2]:
ollama_test_models = {
    "llama-3.1" : "llama3.1:8b-instruct-fp16",
    "llama-3.1:int8" : "llama3.1:8b-instruct-q8_0",
    "llama-3.1:int4" : "llama3.1:8b-instruct-q4_0",
    "qwen-2.5" : "qwen2.5:7b-instruct-fp16",
    "qwen-2.5:int8" : "qwen2.5:7b-instruct-q8_0",
    "qwen-2.5:int4" : "qwen2.5:7b-instruct-q4_K_M",
    "qwen-2.5-14b:int4" : "qwen2.5:14b-instruct-q4_K_M",
    "qwen-2.5-32b:int4" : "qwen2.5:32b-instruct-q4_K_M",
    "qwen-2.5-coder:int8" : "qwen2.5-coder:7b-instruct-q8_0",
    "qwen-2.5-coder:int4" : "qwen2.5-coder:7b-instruct-q4_K_M"
}

## Test prompts

In [3]:
test_messages = [
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages du Crédit Mutuel ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages du Crédit Agricole ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages de la Société Générale ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages de la BNP ?"}
]
]

In [4]:
from transformers import AutoTokenizer

def format_prompt(messages, model):
    tokenizer = AutoTokenizer.from_pretrained(model)
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

In [5]:
format_prompt(test_messages, test_models["qwen-2.5"])

['<|im_start|>system\nTu es un assistant utile et professionnel qui répond toujours en français.<|im_end|>\n<|im_start|>user\nQuels sont les avantages du Crédit Mutuel ?<|im_end|>\n<|im_start|>assistant\n',
 '<|im_start|>system\nTu es un assistant utile et professionnel qui répond toujours en français.<|im_end|>\n<|im_start|>user\nQuels sont les avantages du Crédit Agricole ?<|im_end|>\n<|im_start|>assistant\n',
 '<|im_start|>system\nTu es un assistant utile et professionnel qui répond toujours en français.<|im_end|>\n<|im_start|>user\nQuels sont les avantages de la Société Générale ?<|im_end|>\n<|im_start|>assistant\n',
 '<|im_start|>system\nTu es un assistant utile et professionnel qui répond toujours en français.<|im_end|>\n<|im_start|>user\nQuels sont les avantages de la BNP ?<|im_end|>\n<|im_start|>assistant\n']

### vLLM

In [6]:
# Authenticate VLLM with Huggingface Hub
import os

with open("/workspace/hftoken", 'r') as file:
    myhftoken = file.read().strip()

os.environ["HF_TOKEN"]=myhftoken

In [7]:
import time
from vllm import LLM, SamplingParams

def vllm_load(model):    
    llm = LLM(model, gpu_memory_utilization=0.99, max_model_len=8192)
    llm._model = model
    return llm

def vllm_generate(messages, llm):    
    print(f"vLLM performance test:")
    
    prompts = format_prompt(messages, llm._model)
    sampling_params = SamplingParams(temperature=0.7, top_p=0.8, repetition_penalty=1.05, max_tokens=512)
    # warmup
    outputs = llm.generate(prompts[0], sampling_params)
    print(f"Generated text: {outputs[0].outputs[0].text!r}")
    
    for batch_size in range(1, len(messages) + 1):
        start_time = time.time()  # Record the start time
        outputs = llm.generate(prompts[0:batch_size], sampling_params)
        end_time = time.time()  # Record the end time
            
        # Print the outputs.
        tokenscount = 0
        for output in outputs:
            generated_text = output.outputs[0].text
            tokenscount = tokenscount + len(output.outputs[0].token_ids)

        tokens_per_sec = tokenscount/(end_time-start_time)
        print(f"- batch size {batch_size}: {tokens_per_sec:.2f} tokens/sec ({batch_size} x {tokens_per_sec/batch_size:.2f})")

In [ ]:
llm = vllm_load(test_models["llama-3.1"])

In [ ]:
vllm_generate(test_messages*8, llm)

### SGLang

In [6]:
import json, time
import sglang

def sglang_load(model):
    runtime = sglang.Runtime(model_path=model)
    runtime._model = model
    return runtime

def sglang_generate(messages, runtime):
    print(f"SGLang performance test:")
    
    prompts = format_prompt(messages, runtime._model)
    sampling_params = { "temperature":0.7, "top_p":0.8, "repetition_penalty":1.05, "max_new_tokens":512 }
    # warmup
    output = json.loads(runtime.generate(prompt=prompts[0], sampling_params=sampling_params))
    print(f"Generated text: {output['text']!r}")
    
    for batch_size in range(1, len(messages) + 1):
        start_time = time.time()  # Record the start time
        outputs = json.loads(runtime.generate(prompt=prompts[0:batch_size], sampling_params=sampling_params))
        end_time = time.time()  # Record the end time
            
        # Print the outputs.
        tokenscount = 0
        for output in outputs:
            generated_text = output["text"]
            tokenscount = tokenscount + output["meta_info"]["completion_tokens"]

        tokens_per_sec = tokenscount/(end_time-start_time)
        print(f"- batch size {batch_size}: {tokens_per_sec:.2f} tokens/sec ({batch_size} x {tokens_per_sec/batch_size:.2f})")

In [ ]:
runtime = sglang_load(test_models["llama-3.1"])

In [ ]:
sglang_generate(test_messages*8, runtime)

### Ollama

https://github.com/ollama/ollama-python

In [45]:
import ollama

ollama.list()

{'models': [{'name': 'llama3.1:8b-instruct-q4_0',
   'model': 'llama3.1:8b-instruct-q4_0',
   'modified_at': '2024-09-22T11:43:03.484852591+02:00',
   'size': 4661230766,
   'digest': '42182419e9508c30c4b1fe55015f06b65f4ca4b9e28a744be55008d21998a093',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '8.0B',
    'quantization_level': 'Q4_0'}},
  {'name': 'llama3.1:8b-instruct-q8_0',
   'model': 'llama3.1:8b-instruct-q8_0',
   'modified_at': '2024-09-22T11:33:10.006555339+02:00',
   'size': 8540789934,
   'digest': 'b158ded76fa05be6bce8a682099ce5df8c5571340a04cf63a2923464679db576',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '8.0B',
    'quantization_level': 'Q8_0'}},
  {'name': 'llama3.1:8b-instruct-fp16',
   'model': 'llama3.1:8b-instruct-fp16',
   'modified_at': '2024-09-21T22:54:30.926572546+02:00',
   'size': 16068910253

In [27]:
print(ollama.show("llama3.1:8b-instruct-fp16")['template'])

{{- if or .System .Tools }}<|start_header_id|>system<|end_header_id|>
{{- if .System }}

{{ .System }}
{{- end }}
{{- if .Tools }}

Cutting Knowledge Date: December 2023

When you receive a tool call response, use the output to format an answer to the orginal user question.

You are a helpful assistant with tool calling capabilities.
{{- end }}<|eot_id|>
{{- end }}
{{- range $i, $_ := .Messages }}
{{- $last := eq (len (slice $.Messages $i)) 1 }}
{{- if eq .Role "user" }}<|start_header_id|>user<|end_header_id|>
{{- if and $.Tools $last }}

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}. Do not use variables.

{{ range $.Tools }}
{{- . }}
{{ end }}
Question: {{ .Content }}<|eot_id|>
{{- else }}

{{ .Content }}<|eot_id|>
{{- end }}{{ if $last }}<|start_header_id|>assistant<|end_header_id|>

{{ e

https://github.com/ollama/ollama/blob/main/docs/api.md#generate-a-completion

In [16]:
import ollama
import time

# ollama keeps models 5 min in memory by default, they are reloaded by a query
def ollama_load(model):
    sampling_params = { "num_predict":1 }
    ollama.generate(model=model, prompt="load", raw=True, options=sampling_params, stream=False)
    return model

# ollama API only supports batch size 1
def ollama_generate(messages, model, hftokenizer):
    print(f"ollama performance test:")
    
    prompts = format_prompt(messages, hftokenizer)
    sampling_params = { "temperature":0.7, "top_p":0.8, "repeat_penalty":1.05, "num_predict":512 }
    # warmup
    output = ollama.generate(model=model, prompt=prompts[0], raw=True, options=sampling_params, stream=False)
    print(f"Generated text: {output['response']!r}")
    
    for msg_index in range(len(messages)):
        start_time = time.time()  # Record the start time
        output = ollama.generate(model=model, prompt=prompts[msg_index], raw=True, options=sampling_params, stream=False)
        end_time = time.time()  # Record the end time
            
        # Print the outputs.
        tokenscount = output['eval_count']
        tokens_per_sec = tokenscount/(end_time-start_time)
        print(f"- batch size 1: {tokens_per_sec:.2f} tokens/sec")

In [43]:
model = ollama_load(ollama_test_models["llama-3.1"])

In [ ]:
ollama_generate(test_messages, model, test_models["llama-3.1"])

# 3. Performance tests on RTX 4090 - llama-3.1

## vLLM

### FP16

In [7]:
llm = vllm_load(test_models["llama-3.1"])

INFO 09-22 12:18:56 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=meta-llama/Meta-Llama-3.1-8B-Instruct, use_v2_block_manager=False, num_scheduler_steps=1, en

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 09-22 12:19:07 model_runner.py:1008] Loading model weights took 14.9888 GB
INFO 09-22 12:19:08 gpu_executor.py:122] # GPU blocks: 3610, # CPU blocks: 2048
INFO 09-22 12:19:08 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-22 12:19:08 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-22 12:19:18 model_runner.py:1430] Graph capturing finished in 10 secs.


In [8]:
vllm_generate(test_messages*12, llm)

vLLM performance test:


Processed prompts: 100%|█████████████| 1/1 [00:09<00:00,  9.62s/it, est. speed input: 6.55 toks/s, output: 53.23 toks/s]


Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre plusieurs avantages à ses membres et clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts plus élevés sur les épargnes** : Le Crédit Mutuel propose des taux d'intérêt plus élevés sur les comptes d'épargne que de nombreux autres établissements bancaires.\n2. **Taux d'emprunt compétitifs** : Les prêts personnels, les prêts immobiliers et les prêts pour la mobilité sont proposés avec des taux d'intérêt attractifs.\n3. **Services personnalisés** : Le Crédit Mutuel offre des services personnalisés et adaptés aux besoins de ses membres et clients, grâce à une approche relationnelle et à une connaissance approfondie de leurs situations financières.\n4. **Sécurité et confidentialité** : Le Crédit Mutuel s'engage à protéger les données personnelles et financières de ses membres et clients, conformément aux règles de protection des données.\n5. **Participation aux décisions** : En tant que membre 

Processed prompts: 100%|█████████████| 1/1 [00:09<00:00,  9.29s/it, est. speed input: 6.78 toks/s, output: 55.11 toks/s]


- batch size 1: 55.10 tokens/sec (1 x 55.10)


...

- batch size 46: 1070.79 tokens/sec (46 x 23.28)


Processed prompts: 100%|███████| 47/47 [00:22<00:00,  2.13it/s, est. speed input: 134.17 toks/s, output: 1087.55 toks/s]


- batch size 47: 1087.21 tokens/sec (47 x 23.13)


Processed prompts: 100%|███████| 48/48 [00:22<00:00,  2.13it/s, est. speed input: 134.46 toks/s, output: 1085.58 toks/s]

- batch size 48: 1085.12 tokens/sec (48 x 22.61)


### w8a16

In [8]:
llm = vllm_load(test_models["llama-3.1:w8a16"])

INFO 09-22 12:47:04 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a16', speculative_config=None, tokenizer='neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a16', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=compressed-tensors, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=neuralmagic/Meta-Llama-3.1-8B-Instruct-quant

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 09-22 12:47:08 model_runner.py:1008] Loading model weights took 8.4927 GB
INFO 09-22 12:47:09 gpu_executor.py:122] # GPU blocks: 6716, # CPU blocks: 2048
INFO 09-22 12:47:09 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-22 12:47:09 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-22 12:47:18 model_runner.py:1430] Graph capturing finished in 9 secs.


In [10]:
vllm_generate(test_messages*18, llm)

vLLM performance test:


Processed prompts: 100%|█████████████| 1/1 [00:06<00:00,  6.05s/it, est. speed input: 6.94 toks/s, output: 84.63 toks/s]


Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre de nombreux avantages à ses adhérents et clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts plus élevés** : Le Crédit Mutuel propose des taux d'intérêt plus élevés que les banques traditionnelles, notamment pour les comptes courants et les prêts.\n2. **Services personnalisés** : En tant que banque coopérative, le Crédit Mutuel met l'accent sur la proximité et la personnalisation de ses services. Les clients ont accès à des conseillers financiers compétents qui leur proposent des solutions adaptées à leurs besoins.\n3. **Transparence et sécurité** : Le Crédit Mutuel est connu pour sa transparence dans les tarifs et les conditions de prêt. Les clients sont également protégés par des garanties de sécurité renforcées.\n4. **Épargne et placement** : Le Crédit Mutuel propose une gamme de produits d'épargne et de placement attractifs, notamment des comptes d'épargne, des livrets et des fonds 

Processed prompts: 100%|█████████████| 1/1 [00:05<00:00,  5.82s/it, est. speed input: 7.21 toks/s, output: 87.92 toks/s]


- batch size 1: 87.89 tokens/sec (1 x 87.89)


...

- batch size 67: 2457.61 tokens/sec (67 x 36.68)


Processed prompts: 100%|███████| 68/68 [00:14<00:00,  4.75it/s, est. speed input: 199.54 toks/s, output: 2413.53 toks/s]


- batch size 68: 2412.29 tokens/sec (68 x 35.47)


Processed prompts:   0%|                     | 0/69 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

KeyboardInterrupt: 

### fp8

Note: the "FP8-dynamic" version is much slower that the "FP8" version.
- batch size 16: 61 tokens/sec vs 77 tokens/sec
- batch size 32: 42 tokens/sec vs 62 tokens/sec
- batch size 48: 03 tokens/sec vs 52 tokens/sec !

In [8]:
llm = vllm_load(test_models["llama-3.1:fp8"])

config.json:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

INFO 09-22 13:39:33 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='neuralmagic/Meta-Llama-3.1-8B-Instruct-FP8', speculative_config=None, tokenizer='neuralmagic/Meta-Llama-3.1-8B-Instruct-FP8', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=compressed-tensors, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=neuralmagic/Meta-Llama-3.1-8B-Instruct-FP8, use_v2_block_manager=Fal

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/62.8k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 09-22 13:47:57 model_runner.py:1008] Loading model weights took 8.4889 GB
INFO 09-22 13:47:58 gpu_executor.py:122] # GPU blocks: 6856, # CPU blocks: 2048
INFO 09-22 13:47:58 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-22 13:47:58 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-22 13:48:07 model_runner.py:1430] Graph capturing finished in 9 secs.


In [9]:
vllm_generate(test_messages*16, llm)

vLLM performance test:


Processed prompts: 100%|████████████| 1/1 [00:06<00:00,  6.11s/it, est. speed input: 10.32 toks/s, output: 83.84 toks/s]


Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre plusieurs avantages à ses membres et clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts plus élevés sur les épargnes** : Le Crédit Mutuel propose des taux d'intérêt plus élevés sur les comptes d'épargne comparés aux banques traditionnelles.\n2. **Frais de gestion réduits** : Les frais de gestion pour les comptes courants et les prêts sont souvent moins élevés chez le Crédit Mutuel que dans les banques conventionnelles.\n3. **Prêts personnalisés** : Le Crédit Mutuel prend en compte la situation personnelle et financière de chaque client pour proposer des prêts adaptés à ses besoins.\n4. **Soutien à l'entrepreneuriat** : Le Crédit Mutuel propose des solutions financières spécifiques pour les entrepreneurs et les PME, notamment des prêts et des services de financement.\n5. **Services bancaires complets** : Le Crédit Mutuel propose une gamme complète de services bancaires, y compris les co

Processed prompts: 100%|████████████| 1/1 [00:05<00:00,  5.93s/it, est. speed input: 10.62 toks/s, output: 86.31 toks/s]


- batch size 1: 86.27 tokens/sec (1 x 86.27)


...

- batch size 65: 972.07 tokens/sec (65 x 14.95)


Processed prompts: 100%|████████| 66/66 [00:34<00:00,  1.92it/s, est. speed input: 120.88 toks/s, output: 978.95 toks/s]


- batch size 66: 978.71 tokens/sec (66 x 14.83)


Processed prompts:   0%|                     | 0/67 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

KeyboardInterrupt: 

### w8a8

In [8]:
llm = vllm_load(test_models["llama-3.1:w8a8"])

config.json:   0%|          | 0.00/2.14k [00:00<?, ?B/s]

INFO 09-22 14:08:30 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a8', speculative_config=None, tokenizer='neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a8', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=compressed-tensors, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=neuralmagic/Meta-Llama-3.1-8B-Instruct-quantiz

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/43.5k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 09-22 14:16:47 model_runner.py:1008] Loading model weights took 8.4939 GB
INFO 09-22 14:16:48 gpu_executor.py:122] # GPU blocks: 6759, # CPU blocks: 2048
INFO 09-22 14:16:48 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-22 14:16:48 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-22 14:16:57 model_runner.py:1430] Graph capturing finished in 9 secs.


In [9]:
vllm_generate(test_messages*18, llm)

vLLM performance test:


Processed prompts: 100%|█████████████| 1/1 [00:06<00:00,  6.90s/it, est. speed input: 6.09 toks/s, output: 74.24 toks/s]


Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre plusieurs avantages à ses membres et clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts plus faibles** : Le Crédit Mutuel propose des taux d'intérêt compétitifs pour les prêts, les comptes courants et les épargnes.\n2. **Accès à des services bancaires complets** : Le Crédit Mutuel offre une gamme complète de services bancaires, y compris des prêts, des comptes courants, des épargnes, des cartes de crédit, des assurances et des investissements.\n3. **Conseils personnalisés** : Les conseillers du Crédit Mutuel peuvent vous aider à gérer votre patrimoine, à élaborer un plan financier et à atteindre vos objectifs économiques.\n4. **Transparence et sécurité** : Le Crédit Mutuel est une banque coopérative, ce qui signifie que les décisions sont prises collectivement par les membres et non par des actionnaires. Cela assure une transparence et une sécurité supplémentaires.\n5. **Services en li

Processed prompts: 100%|█████████████| 1/1 [00:06<00:00,  6.63s/it, est. speed input: 6.33 toks/s, output: 77.21 toks/s]


- batch size 1: 77.18 tokens/sec (1 x 77.18)


...

- batch size 31: 337.58 tokens/sec (31 x 10.89)


Processed prompts: 100%|█████████| 32/32 [00:59<00:00,  1.87s/it, est. speed input: 22.40 toks/s, output: 269.94 toks/s]


- batch size 32: 269.92 tokens/sec (32 x 8.44)


Processed prompts:   0%|                     | 0/33 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

KeyboardInterrupt: 

### w4a16

In [8]:
llm = vllm_load(test_models["llama-3.1:w4a16"])

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

INFO 09-22 14:25:08 gptq_marlin.py:108] The model is convertible to gptq_marlin during runtime. Using gptq_marlin kernel.
INFO 09-22 14:25:08 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w4a16', speculative_config=None, tokenizer='neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w4a16', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_tim

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

WARNING 09-22 14:25:10 utils.py:727] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
INFO 09-22 14:25:10 model_runner.py:997] Starting to load model neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w4a16...
INFO 09-22 14:25:11 weight_utils.py:242] Using model weights format ['*.safetensors']


model.safetensors:   0%|          | 0.00/5.74G [00:00<?, ?B/s]

INFO 09-22 14:30:25 weight_utils.py:287] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 09-22 14:30:26 model_runner.py:1008] Loading model weights took 5.3812 GB
INFO 09-22 14:30:28 gpu_executor.py:122] # GPU blocks: 8325, # CPU blocks: 2048
INFO 09-22 14:30:30 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-22 14:30:30 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-22 14:30:40 model_runner.py:1430] Graph capturing finished in 10 secs.


In [9]:
vllm_generate(test_messages*18, llm)

vLLM performance test:


Processed prompts: 100%|████████████| 1/1 [00:04<00:00,  4.38s/it, est. speed input: 9.58 toks/s, output: 116.78 toks/s]


Generated text: "Le Crédit Mutuel est une banque mutuelle française qui offre divers avantages à ses clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts sur les comptes courants** : Le Crédit Mutuel offre des intérêts sur les comptes courants, ce qui permet aux clients de gagner de l'argent sans avoir à investir leurs fonds.\n2. **Prêt à usage** : Le Crédit Mutuel propose des prêts à usage avec des taux d'intérêt compétitifs, ce qui peut aider les clients à financer leurs besoins ou leurs projets.\n3. **Assurance de prêt** : Le Crédit Mutuel offre une assurance de prêt qui couvre les risques de défaut de remboursement, ce qui donne aux emprunteurs une sécurité supplémentaire.\n4. **Services bancaires complets** : Le Crédit Mutuel propose une gamme complète de services bancaires, y compris des cartes de crédit, des chèques, des virements et des paiements en ligne.\n5. **Sécurité et protection** : Le Crédit Mutuel met en place des mesures de sécurité pour protéger les

Processed prompts: 100%|███████████| 1/1 [00:04<00:00,  4.08s/it, est. speed input: 10.30 toks/s, output: 125.62 toks/s]


- batch size 1: 125.55 tokens/sec (1 x 125.55)


...

- batch size 70: 2874.84 tokens/sec (70 x 41.07)


Processed prompts: 100%|███████| 71/71 [00:11<00:00,  5.99it/s, est. speed input: 251.64 toks/s, output: 2927.52 toks/s]


- batch size 71: 2925.52 tokens/sec (71 x 41.20)


Processed prompts: 100%|███████| 72/72 [00:12<00:00,  5.91it/s, est. speed input: 248.35 toks/s, output: 2909.87 toks/s]

- batch size 72: 2907.97 tokens/sec (72 x 40.39)


## SGLang

### fp16

In [7]:
runtime = sglang_load(test_models["llama-3.1"])

INFO 09-22 14:43:27 weight_utils.py:242] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


In [8]:
sglang_generate(test_messages*8, runtime)

SGLang performance test:
Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre divers avantages à ses membres, notamment :\n\n1. **Intérêts plus élevés sur les épargnes** : Les membres du Crédit Mutuel bénéficient souvent de taux d'intérêt plus élevés que ceux proposés par les banques traditionnelles sur leurs comptes d'épargne.\n2. **Prêts à des conditions favorables** : Les membres peuvent obtenir des prêts à des taux d'intérêt attractifs et avec des conditions de remboursement flexibles.\n3. **Sécurité et stabilité** : En tant que banque coopérative, le Crédit Mutuel est contrôlée par ses membres, ce qui garantit une gestion responsable et durable de l'argent.\n4. **Services personnalisés** : Les conseillers du Crédit Mutuel prennent le temps de comprendre les besoins de chaque membre pour offrir des solutions adaptées à leur situation financière.\n5. **Économies sur les frais** : Les membres du Crédit Mutuel peuvent bénéficier de réductions sur les frais 

### w8a16

In [7]:
runtime = sglang_load(test_models["llama-3.1:w8a16"])

INFO 09-22 14:52:36 weight_utils.py:242] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Process Process-1:2:
Traceback (most recent call last):
  File "/root/miniconda3/envs/wordslab-notebooks/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/root/miniconda3/envs/wordslab-notebooks/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/workspace/wordslab-llms/.venv/lib/python3.11/site-packages/sglang/srt/managers/detokenizer_manager.py", line 185, in start_detokenizer_process
    loop.run_until_complete(manager.handle_loop())
  File "uvloop/loop.pyx", line 1511, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1504, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1377, in uvloop.loop.Loop.run_forever
  File "uvloop/loop.pyx", line 555, in uvloop.loop.Loop._run
Process Process-1:1:
Traceback (most recent call last):
  File "/root/miniconda3/envs/wordslab-notebooks/lib/python3.11/multiprocessing/process.py", line 314, in _bootstra

In [8]:
sglang_generate(test_messages*10, runtime)

SGLang performance test:
Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre plusieurs avantages à ses adhérents et clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts attractifs** : Le Crédit Mutuel propose des taux d'intérêt compétitifs pour les prêts, les comptes courants et les épargnes.\n2. **Services personnalisés** : En tant que banque coopérative, le Crédit Mutuel se concentre sur la relation avec ses adhérents et clients, offrant des services personnalisés et adaptés à leurs besoins.\n3. **Sécurité et confidentialité** : Comme banque coopérative, le Crédit Mutuel est soumis à des règles de confidentialité et de sécurité strictes pour protéger les données personnelles et financières de ses adhérents et clients.\n4. **Prêts sans frais de dossier** : Le Crédit Mutuel propose des prêts sans frais de dossier pour certains types de prêts, ce qui peut aider les clients à économiser de l'argent.\n5. **Cartes de crédit et de débit** : Le

KeyboardInterrupt: 

### fp8

In [ ]:
runtime = sglang_load(test_models["llama-3.1:fp8"])

**INTERNAL ERROR** during FP8 model load

### w8a8

In [7]:
runtime = sglang_load(test_models["llama-3.1:w8a8"])

INFO 09-22 15:04:44 weight_utils.py:242] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Process Process-1:2:
Traceback (most recent call last):
  File "/root/miniconda3/envs/wordslab-notebooks/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/root/miniconda3/envs/wordslab-notebooks/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/workspace/wordslab-llms/.venv/lib/python3.11/site-packages/sglang/srt/managers/detokenizer_manager.py", line 185, in start_detokenizer_process
    loop.run_until_complete(manager.handle_loop())
  File "uvloop/loop.pyx", line 1511, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1504, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1377, in uvloop.loop.Loop.run_forever
  File "uvloop/loop.pyx", line 555, in uvloop.loop.Loop._run
  File "uvloop/handles/poll.pyx", line 216, in uvloop.loop.__on_uvpoll_event
  File "uvloop/cbhandles.pyx", line 83, in uvloop.loop.Handle._run
  File "uvloop/cbhandles.py

In [8]:
sglang_generate(test_messages*8, runtime)

SGLang performance test:
Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre divers avantages à ses adhérents et clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts attractifs** : Le Crédit Mutuel propose des taux d'intérêt compétitifs pour les prêts, les comptes courants et les épargnes.\n2. **Service personnalisé** : En tant que banque coopérative, le Crédit Mutuel se concentre sur la relation avec ses clients et adhérents, offrant un service personnalisé et attentif.\n3. **Produits financiers adaptés** : Le Crédit Mutuel propose une large gamme de produits financiers adaptés aux besoins de ses clients, tels que les prêts immobiliers, les prêts personnels, les comptes courants, les comptes d'épargne, etc.\n4. **Sécurité et protection** : Le Crédit Mutuel met en place des mesures de sécurité et de protection pour protéger les données et les comptes de ses clients.\n5. **Transparence et responsabilité** : En tant que banque coopérative, 

KeyboardInterrupt: 

### w4a16

In [7]:
runtime = sglang_load(test_models["llama-3.1:w4a16"])

INFO 09-22 15:13:32 gptq_marlin.py:108] The model is convertible to gptq_marlin during runtime. Using gptq_marlin kernel.
INFO 09-22 15:13:33 weight_utils.py:242] Using model weights format ['*.safetensors']
INFO 09-22 15:13:33 weight_utils.py:287] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Process Process-1:2:
Traceback (most recent call last):
  File "/root/miniconda3/envs/wordslab-notebooks/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/root/miniconda3/envs/wordslab-notebooks/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/workspace/wordslab-llms/.venv/lib/python3.11/site-packages/sglang/srt/managers/detokenizer_manager.py", line 185, in start_detokenizer_process
    loop.run_until_complete(manager.handle_loop())
  File "uvloop/loop.pyx", line 1511, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1504, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1377, in uvloop.loop.Loop.run_forever
  File "uvloop/loop.pyx", line 555, in uvloop.loop.Loop._run
  File "uvloop/handles/poll.pyx", line 216, in uvloop.loop.__on_uvpoll_event
  File "uvloop/cbhandles.pyx", line 83, in uvloop.loop.Handle._run
  File "uvloop/cbhandles.py

In [8]:
sglang_generate(test_messages*18, runtime)

SGLang performance test:
Generated text: "Le Crédit Mutuel est une banque mutuelle française qui offre divers services financiers à ses clients. Voici quelques-uns des principaux avantages du Crédit Mutuel :\n\n1. **Sécurité et stabilité** : Le Crédit Mutuel est une banque mutuelle, ce qui signifie qu'il est géré par ses membres (clients) et non par des actionnaires. Cela garantit sa stabilité et sa sécurité.\n2. **Services personnalisés** : Le Crédit Mutuel propose des services personnalisés adaptés aux besoins de ses clients, qu'ils soient particuliers ou professionnels.\n3. **Offres de produits variées** : La banque propose une gamme de produits et services, notamment des comptes courants, des épargnes, des crédits, des assurances et des services d'investissement.\n4. **Prêt rapide et à bon taux d'intérêt** : Le Crédit Mutuel offre des prêts à des taux d'intérêt compétitifs et avec des conditions de remboursement flexibles.\n5. **Assistance et conseil** : Les clients du Crédit Mutue

KeyboardInterrupt: 

## ollama

### fp16

In [11]:
model = ollama_load(ollama_test_models["llama-3.1"])

In [17]:
ollama_generate(test_messages, model, test_models["llama-3.1"])

ollama performance test:
Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre plusieurs avantages à ses membres. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts plus élevés sur les épargnes** : Les membres du Crédit Mutuel bénéficient souvent d'intérêts plus élevés sur leurs épargnes comparés aux banques traditionnelles.\n2. **Prêt à taux réduit** : Le Crédit Mutuel propose des prêts à taux réduits pour les membres, ce qui peut être avantageux pour les personnes qui cherchent à financer une acquisition ou un projet.\n3. **Services personnalisés** : Les conseillers du Crédit Mutuel sont souvent plus accessibles et plus proches de leurs clients que ceux des banques traditionnelles, ce qui permet un service personnalisé et plus adapté aux besoins de chaque membre.\n4. **Participation aux bénéfices** : Les membres du Crédit Mutuel participent aux bénéfices de l'organisme, ce qui peut leur permettre de bénéficier de dividendes ou de primes.\n5. **S

### int8

In [18]:
model = ollama_load(ollama_test_models["llama-3.1:int8"])

In [19]:
ollama_generate(test_messages, model, test_models["llama-3.1"])

ollama performance test:
Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre divers avantages à ses membres et clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts plus élevés sur les comptes courants** : Le Crédit Mutuel propose des taux d'intérêt plus élevés que les banques traditionnelles pour les comptes courants, ce qui peut aider les clients à gagner de l'argent en fonction de leurs dépôts.\n2. **Services personnalisés** : En tant que banque coopérative, le Crédit Mutuel se concentre sur la relation client et offre des services personnalisés pour répondre aux besoins de ses membres et clients.\n3. **Prêts à taux réduits** : Le Crédit Mutuel propose des prêts à des taux d'intérêt réduits, ce qui peut aider les clients à acheter une maison ou à financer un projet important.\n4. **Assurance de crédit groupé** : Le Crédit Mutuel offre une assurance de crédit groupé qui permet aux clients de bénéficier d'une protection en cas de difficul

### int4

In [20]:
model = ollama_load(ollama_test_models["llama-3.1:int4"])

In [21]:
ollama_generate(test_messages, model, test_models["llama-3.1"])

ollama performance test:
Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre diverses avantages à ses adhérents et clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts plus élevés** : Le Crédit Mutuel propose des taux d'intérêt plus attractifs que les banques traditionnelles pour les comptes courants, les prêts et les épargnes.\n2. **Système coopératif** : En tant que banque coopérative, le Crédit Mutuel est géré par ses adhérents, qui sont également des actionnaires. Cela signifie que les décisions stratégiques sont prises collectivement, ce qui peut conduire à des avantages pour les membres.\n3. **Services personnalisés** : Le Crédit Mutuel se préoccupe de la personnalisation des services bancaires pour répondre aux besoins spécifiques de ses clients.\n4. **Épargne et épargne à long terme** : Le Crédit Mutuel propose des produits d'épargne à long terme, tels que les livrets et les comptes d'épargne retraite, qui peuvent aider les client

# 4. Performance tests on RTX 4090 - qwen-2.5

## vLLM

### FP16

In [9]:
llm = vllm_load(test_models["qwen-2.5"])

INFO 09-23 23:09:00 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='Qwen/Qwen2.5-7B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=Qwen/Qwen2.5-7B-Instruct, use_v2_block_manager=False, num_scheduler_steps=1, enable_prefix_caching=False, use_async_ou

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 09-23 23:09:13 model_runner.py:1008] Loading model weights took 14.2487 GB
INFO 09-23 23:09:15 gpu_executor.py:122] # GPU blocks: 8760, # CPU blocks: 4681
INFO 09-23 23:09:16 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-23 23:09:16 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-23 23:09:25 model_runner.py:1430] Graph capturing finished in 9 secs.


In [10]:
vllm_generate(test_messages*12, llm)

vLLM performance test:


Processed prompts: 100%|█████████████| 1/1 [00:08<00:00,  8.84s/it, est. speed input: 4.52 toks/s, output: 57.91 toks/s]


Generated text: "Le Crédit Mutuel est une coopérative financière qui propose divers produits et services bancaires. Voici quelques-uns des principaux avantages qu'il offre :\n\n1. **Structure Coopérative** : Le Crédit Mutuel est une coopérative de crédit, ce qui signifie que les clients sont également propriétaires. Cette structure permet une plus grande transparence et responsabilité vis-à-vis des membres.\n\n2. **Service Client Qualifié** : Les conseillers du Crédit Mutuel sont souvent connus pour leur expertise et leur disponibilité. Ils peuvent offrir un service personnalisé et adaptatif selon les besoins spécifiques de chaque client.\n\n3. **Large Gamme de Produits Financiers** : Le Crédit Mutuel propose une large gamme de produits financiers, y compris des comptes courants, des épargnes, des prêts immobiliers, des cartes de crédit, des assurances et des placements.\n\n4. **Réseau de Bureaux Étendu** : Avec un réseau de succursales étendu dans toute la France, le Crédit Mutuel off

Processed prompts: 100%|█████████████| 1/1 [00:08<00:00,  8.67s/it, est. speed input: 4.62 toks/s, output: 58.74 toks/s]


- batch size 1: 58.72 tokens/sec (1 x 58.72)


...

- batch size 46: 1169.62 tokens/sec (46 x 25.43)


Processed prompts: 100%|█████████| 47/47 [00:24<00:00,  1.90it/s, est. speed input: 75.92 toks/s, output: 864.81 toks/s]


- batch size 47: 864.60 tokens/sec (47 x 18.40)


Processed prompts: 100%|█████████| 48/48 [00:25<00:00,  1.86it/s, est. speed input: 74.53 toks/s, output: 832.82 toks/s]

- batch size 48: 832.64 tokens/sec (48 x 17.35)


### w8a16

In [8]:
llm = vllm_load(test_models["qwen-2.5:w8a16"])

INFO 09-23 23:25:51 gptq_marlin.py:108] The model is convertible to gptq_marlin during runtime. Using gptq_marlin kernel.
INFO 09-23 23:25:51 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='Qwen/Qwen2.5-7B-Instruct-GPTQ-Int8', speculative_config=None, tokenizer='Qwen/Qwen2.5-7B-Instruct-GPTQ-Int8', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=Fals

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 09-23 23:26:07 model_runner.py:1008] Loading model weights took 8.3072 GB
INFO 09-23 23:26:09 gpu_executor.py:122] # GPU blocks: 15179, # CPU blocks: 4681
INFO 09-23 23:26:13 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-23 23:26:13 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-23 23:26:23 model_runner.py:1430] Graph capturing finished in 10 secs.


In [9]:
vllm_generate(test_messages*18, llm)

vLLM performance test:


Processed prompts: 100%|█████████████| 1/1 [00:05<00:00,  5.84s/it, est. speed input: 6.86 toks/s, output: 87.75 toks/s]


Generated text: "Le Crédit Mutuel est une coopérative financière qui propose divers produits et services bancaires. Voici quelques-uns des principaux avantages qu'il offre :\n\n1. **Structure Coopérative** : Le Crédit Mutuel est une coopérative de crédit, ce qui signifie que ses actionnaires sont ses clients. Cette structure permet une gestion plus démocratique et une répartition des bénéfices entre les clients.\n\n2. **Services Personnalisés** : Les conseillers du Crédit Mutuel sont formés pour offrir des conseils personnalisés et adaptés aux besoins spécifiques de chaque client. Cela peut être particulièrement avantageux pour les petites entreprises ou les particuliers ayant des situations financières complexes.\n\n3. **Large Portefeuille de Produits Financiers** : Le Crédit Mutuel propose une gamme étendue de produits financiers, allant des comptes courants et épargne à des prêts immobiliers, des cartes bancaires, des assurances et des placements.\n\n4. **Tarifs Avantageux** : Les m

Processed prompts: 100%|█████████████| 1/1 [00:04<00:00,  4.74s/it, est. speed input: 8.44 toks/s, output: 90.52 toks/s]


- batch size 1: 90.46 tokens/sec (1 x 90.46)


...

- batch size 70: 2522.71 tokens/sec (70 x 36.04)


Processed prompts: 100%|███████| 71/71 [00:12<00:00,  5.48it/s, est. speed input: 219.54 toks/s, output: 2532.34 toks/s]


- batch size 71: 2530.42 tokens/sec (71 x 35.64)


Processed prompts: 100%|███████| 72/72 [00:13<00:00,  5.41it/s, est. speed input: 216.43 toks/s, output: 2560.34 toks/s]

- batch size 72: 2558.47 tokens/sec (72 x 35.53)


### w4a16

In [8]:
llm = vllm_load(test_models["qwen-2.5:w4a16"])

INFO 09-23 23:36:52 gptq_marlin.py:108] The model is convertible to gptq_marlin during runtime. Using gptq_marlin kernel.
INFO 09-23 23:36:52 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4', speculative_config=None, tokenizer='Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=Fals

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 09-23 23:37:08 model_runner.py:1008] Loading model weights took 5.2048 GB
INFO 09-23 23:37:12 gpu_executor.py:122] # GPU blocks: 18623, # CPU blocks: 4681
INFO 09-23 23:37:22 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-23 23:37:22 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-23 23:37:33 model_runner.py:1430] Graph capturing finished in 10 secs.


In [9]:
vllm_generate(test_messages*32, llm)

vLLM performance test:


Processed prompts: 100%|███████████| 1/1 [00:03<00:00,  3.76s/it, est. speed input: 10.64 toks/s, output: 118.64 toks/s]


Generated text: "Le Crédit Mutuel est une institution financière coopérative, ce qui signifie que ses clients sont également ses actionnaires. Voici quelques avantages qu'il peut offrir :\n\n1. **Coopération et solidarité** : En tant que coopérative, le Crédit Mutuel favorise la solidarité entre ses clients et ses employés. Les décisions sont prises de manière collective et les bénéfices sont répartis entre les membres.\n\n2. **Service client personnalisé** : Le Crédit Mutuel propose souvent des services personnalisés adaptés aux besoins spécifiques de chaque client, en particulier pour les clients de longue date.\n\n3. **Transparence** : Les actions du Crédit Mutuel sont cotées sur les marchés financiers, ce qui permet une transparence accrue sur les performances financières de l'institution.\n\n4. **Tarifs avantageux** : Parce que le Crédit Mutuel est une coopérative, il peut proposer des taux d'intérêt plus bas sur les prêts et des commissions moins élevées sur les comptes bancaires

Processed prompts: 100%|███████████| 1/1 [00:02<00:00,  2.99s/it, est. speed input: 13.38 toks/s, output: 137.44 toks/s]


- batch size 1: 137.34 tokens/sec (1 x 137.34)


...

- batch size 73: 2817.01 tokens/sec (73 x 38.59)


Processed prompts: 100%|███████| 74/74 [00:12<00:00,  5.73it/s, est. speed input: 229.12 toks/s, output: 2694.05 toks/s]


- batch size 74: 2692.41 tokens/sec (74 x 36.38)


Processed prompts:   0%|                     | 0/75 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

KeyboardInterrupt: 

### 14 B - 8 bits

In [ ]:
llm = vllm_load(test_models["qwen-2.5-14b:w8a16"])

In [ ]:
vllm_generate(test_messages*18, llm)

### 14 B - 4 bits

In [8]:
llm = vllm_load(test_models["qwen-2.5-14b:w4a16"])

INFO 09-23 23:47:35 gptq_marlin.py:108] The model is convertible to gptq_marlin during runtime. Using gptq_marlin kernel.
INFO 09-23 23:47:35 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='Qwen/Qwen2.5-14B-Instruct-GPTQ-Int4', speculative_config=None, tokenizer='Qwen/Qwen2.5-14B-Instruct-GPTQ-Int4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=Fa

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 09-23 23:47:46 model_runner.py:1008] Loading model weights took 9.3813 GB
INFO 09-23 23:47:48 gpu_executor.py:122] # GPU blocks: 4153, # CPU blocks: 1365
INFO 09-23 23:48:00 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-23 23:48:00 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-23 23:48:11 model_runner.py:1430] Graph capturing finished in 11 secs.


In [9]:
vllm_generate(test_messages*18, llm)

vLLM performance test:


Processed prompts: 100%|█████████████| 1/1 [00:05<00:00,  5.47s/it, est. speed input: 7.31 toks/s, output: 74.38 toks/s]


Generated text: "Le Crédit Mutuel offre plusieurs avantages à ses clients :\n\n1. **Taux d'intérêt compétitifs** : Le Crédit Mutuel propose souvent des taux d'intérêt attractifs pour les prêts, les comptes et les produits d'épargne.\n\n2. **Produits variés** : La banque propose une large gamme de services et produits adaptés aux besoins de chaque client, que ce soit pour l'entrepreneuriat, la gestion de patrimoine, ou encore le financement immobilier.\n\n3. **Réseau étendu** : Avec son réseau de plus de 2000 agences, il est facile de trouver un point de service proche de chez soi ou de son lieu de travail.\n\n4. **Flexibilité et personnalisation** : Les offres du Crédit Mutuel sont généralement très flexibles et peuvent être personnalisées selon les besoins spécifiques du client.\n\n5. **Services digitaux** : L'institut bancaire offre des solutions numériques performantes comme le Crédit Mutuel Mobile et Crédit Mutuel Online, permettant aux clients de gérer leurs comptes où qu'ils soie

Processed prompts: 100%|█████████████| 1/1 [00:04<00:00,  4.43s/it, est. speed input: 9.03 toks/s, output: 80.14 toks/s]


- batch size 1: 80.10 tokens/sec (1 x 80.10)


...

- batch size 64: 782.78 tokens/sec (64 x 12.23)


Processed prompts: 100%|█████████| 65/65 [00:32<00:00,  2.02it/s, est. speed input: 80.73 toks/s, output: 741.98 toks/s]


- batch size 65: 741.81 tokens/sec (65 x 11.41)


Processed prompts:   0%|                     | 0/66 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

KeyboardInterrupt: 

### 32 B - 4 bits

In [8]:
llm = vllm_load(test_models["qwen-2.5-32b:w4a16"])

INFO 09-23 23:58:35 gptq_marlin.py:108] The model is convertible to gptq_marlin during runtime. Using gptq_marlin kernel.
INFO 09-23 23:58:35 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='Qwen/Qwen2.5-32B-Instruct-GPTQ-Int4', speculative_config=None, tokenizer='Qwen/Qwen2.5-32B-Instruct-GPTQ-Int4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=Fa

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


INFO 09-24 00:01:20 model_runner.py:1008] Loading model weights took 18.1477 GB
INFO 09-24 00:01:26 gpu_executor.py:122] # GPU blocks: 786, # CPU blocks: 1024
INFO 09-24 00:01:31 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-24 00:01:31 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-24 00:01:44 model_runner.py:1430] Graph capturing finished in 13 secs.


In [11]:
vllm_generate(test_messages*18, llm)

vLLM performance test:


Processed prompts: 100%|█████████████| 1/1 [00:10<00:00, 10.67s/it, est. speed input: 3.75 toks/s, output: 40.86 toks/s]


Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre plusieurs avantages à ses clients :\n\n1. **Service personnalisé** : En tant que banque coopérative, le Crédit Mutuel met l'accent sur le service client de proximité. Les conseillers sont généralement bien formés pour comprendre les besoins spécifiques des clients.\n\n2. **Tarification avantageuse** : Le Crédit Mutuel propose souvent des tarifs compétitifs pour ses produits et services, ce qui peut être particulièrement intéressant pour les emprunts immobiliers ou les placements financiers.\n\n3. **Engagement sociétal et environnemental** : La banque s'engage dans diverses initiatives pour soutenir l'économie locale, promouvoir la durabilité et le développement durable. Cela peut être important pour les clients soucieux de ces questions.\n\n4. **Réseau étendu** : Avec plus de 6000 agences réparties en France, le Crédit Mutuel offre une grande couverture géographique, facilitant ainsi l'accès aux services b

Processed prompts: 100%|█████████████| 1/1 [00:09<00:00,  9.83s/it, est. speed input: 4.07 toks/s, output: 42.82 toks/s]


- batch size 1: 42.81 tokens/sec (1 x 42.81)


...

WARNING 09-24 00:17:33 scheduler.py:1362] Sequence group 1219 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=201


Processed prompts: 100%|█████████| 49/49 [00:25<00:00,  1.93it/s, est. speed input: 77.34 toks/s, output: 768.74 toks/s]


- batch size 49: 768.52 tokens/sec (49 x 15.68)


Processed prompts:   0%|                     | 0/50 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

KeyboardInterrupt: 

## SGLang

### fp16

In [ ]:
runtime = sglang_load(test_models["llama-3.1"])

In [ ]:
sglang_generate(test_messages*8, runtime)

### w8a16

In [ ]:
runtime = sglang_load(test_models["llama-3.1:w8a16"])

In [ ]:
sglang_generate(test_messages*10, runtime)

In [ ]:
sglang_generate(test_messages*8, runtime)

### w4a16

In [ ]:
runtime = sglang_load(test_models["llama-3.1:w4a16"])

In [ ]:
sglang_generate(test_messages*18, runtime)

## ollama

### fp16

In [11]:
model = ollama_load(ollama_test_models["llama-3.1"])

In [ ]:
ollama_generate(test_messages, model, test_models["llama-3.1"])

### int8

In [18]:
model = ollama_load(ollama_test_models["llama-3.1:int8"])

In [ ]:
ollama_generate(test_messages, model, test_models["llama-3.1"])

### int4

In [20]:
model = ollama_load(ollama_test_models["llama-3.1:int4"])

In [ ]:
ollama_generate(test_messages, model, test_models["llama-3.1"])

# Installing Open WebUI

https://docs.openwebui.com/

```bash
docker run -d -p 8000:8080 --gpus all --add-host=host.docker.internal:host-gateway -v /workspace/open-webui:/app/backend/data --name open-webui -e WEBUI_AUTH='False' --restart always ghcr.io/open-webui/open-webui:cuda
```

# Models to explore

- Qwen/Qwen2-VL-7B-Instruct
- Qwen/Qwen2-VL-7B-Instruct-GPTQ-Int8
- Qwen/Qwen2-VL-7B-Instruct-GPTQ-Int4
²---
- unsloth/Mistral-Small-Instruct-2409-bnb-4bit [ready]
- bartowski/Mistral-Small-Instruct-2409-GGUF
---
- mistralai/Pixtral-12B-2409
- DewEfresh/pixtral-12b-8bit

In [3]:
vllm_load("Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4")

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

INFO 09-22 20:19:44 gptq_marlin.py:108] The model is convertible to gptq_marlin during runtime. Using gptq_marlin kernel.
INFO 09-22 20:19:44 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4', speculative_config=None, tokenizer='Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=Fals

tokenizer_config.json:   0%|          | 0.00/7.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/75.4k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 09-22 20:24:59 model_runner.py:1008] Loading model weights took 5.2048 GB
INFO 09-22 20:25:00 gpu_executor.py:122] # GPU blocks: 18623, # CPU blocks: 4681
INFO 09-22 20:25:00 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-22 20:25:00 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-22 20:25:09 model_runner.py:1430] Graph capturing finished in 9 secs.
